In [1]:
!pip install evaluate langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [1]:
import os
import time
# from dotenv import load_dotenv # No longer needed if using Colab Secrets
import google.generativeai as genai
import datasets
import evaluate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata # Import userdata

def generate_summary_with_gemini(prompt_text: str) -> str:
    """주어진 프롬프트 텍스트를 기반으로 Gemini 모델을 사용해 요약문을 생성합니다."""
    try:
        # 사용할 모델 설정
        model = genai.GenerativeModel('gemini-1.5-flash')

        # 모델에 콘텐츠 생성 요청
        response = model.generate_content(prompt_text)

        # 결과 텍스트 반환
        return response.text
    except Exception as e:
        # API 호출 중 에러 발생 시
        return f"요약 생성 중 오류가 발생했습니다: {e}"

In [4]:
def main(news_title: str, news_document: str):
    """메인 실행 함수: API 설정, 프롬프트 정의, 요약 생성 및 결과 출력"""
    # ⚠️ 중요: 이곳에 발급받은 본인의 API 키를 입력하세요.
    print("🔑 Google API 키를 로드합니다...")
    # Load API key from Colab Secrets
    google_api_key = userdata.get('GOOGLE_API_KEY')

    if not google_api_key:
        print("API 키가 설정되지 않았습니다. Colab의 Secrets Manager에 'GOOGLE_API_KEY'를 설정해주세요.")
        return
    genai.configure(api_key=google_api_key)

    # 1. 프롬프트 설계
    prompt_1 = {
        "name": "뉴스 에디터' (간결하고 핵심적인 요약)",
        "text": """
당신은 뉴스 기사에서 핵심 사실만 추출하는 '정보 요약가'입니다. 다음 뉴스 기사의 제목과 본문을 읽고, 아래 조건에 따라 핵심 내용을 3~4개의 불렛포인트로 요약해 주세요.

[요약 조건]
- 기사에 명시된 객관적인 사실만 포함하세요.
- 추측, 의견, 감정적인 표현은 절대 사용하지 마세요.
- 각 문장은 핵심 내용을 담아 간결하게 작성하세요.
- 독자가 기사의 핵심 팩트를 30초 안에 파악할 수 있어야 합니다.
"""
    }

    prompt_2 = {
        "name": "정보 큐레이터(정보를 흥미롭게 재구성)",
        "text": """
당신은 독자들의 호기심을 자극하는 정보 큐레이터입니다. 다음 뉴스 기사에서 놓치지 말아야 할 핵심 포인트를 발췌하여 독자가 흥미를 느끼고 더 궁금해할 만한 요약문을 작성해주세요.

다음 조건에 따라 요약하세요:
- 각 뉴스 기사의 제목과 함께, 가장 중요한 사실 3~4가지를 압축하여 요약하세요.
- 단순히 요약하는 것을 넘어, 독자의 흥미를 유발할 수 있는 매력적인 표현을 사용해보세요. (단, 과장되거나 허위 사실 포함 금지)
- 요약 내용은 기사 본문의 사실에 기반하되, 독자에게 '왜 이 뉴스를 알아야 하는가'에 대한 이유를 간접적으로 제시해 주세요.
- 각 요약은 짧은 문단 또는 핵심 문장 위주의 불릿 포인트 형식으로 구성할 수 있습니다.

출력 형식:

---

### 📰 {뉴스 제목}
- [핵심 포인트 1]
- [핵심 포인트 2]
- [핵심 포인트 3]
- [핵심 포인트 4 (필요 시)]

---
"""
    }

    prompt_3 = {
        "name": "사용자 맞춤형 뉴스 브리퍼' (핵심 정보와 간략한 배경 제공)",
        "text": """
당신은 바쁜 사용자를 위한 맞춤형 뉴스 브리퍼입니다. 다음 뉴스 기사의 주요 내용을 신속하게 전달하고, 사용자가 이 뉴스를 이해하는 데 필요한 핵심 정보를 간략히 제공해주세요.

요약 가이드라인:
- 각 뉴스 제목 아래에 핵심 내용을 3~4문장으로 요약합니다.
- 요약문에는 기사의 핵심 사실과 더불어, 해당 내용이 가지는 의미나 배경을 간략히 언급하여 사용자의 이해를 돕습니다. (단, 해석이나 의견은 지양하고 사실 기반으로만 설명)
- 핵심 키워드를 포함하여 정보 전달력을 높여주세요.
- 문장은 명료하고 전문적인 톤으로 작성하세요.

출력 형식:

---

### 📰 {뉴스 제목}
{요약문}

---

"""
    }
    prompts = [prompt_1, prompt_2, prompt_3]

    # 2. 모델 입력 구성
    full_news_text = f"제목: {news_title}\n본문: {news_document}"

    # 3. 프롬프트별 요약 생성 및 비교 출력
    print("=" * 80)
    print("동일 뉴스 기사에 대한 프롬프트별 요약 결과 비교")
    print(f"사용 모델: gemini-1.5-flash")
    print("=" * 80)

    for i, p_info in enumerate(prompts):
        print(f"\n--- [ 프롬프트 #{i+1}: {p_info['name']} ] ---\n")

        # 프롬프트와 뉴스 기사 결합
        input_text = f"{p_info['text']}\n\n[뉴스 기사]\n{full_news_text}"

        # 요약 생성
        summary_result = generate_summary_with_gemini(input_text)

        # 결과 출력
        print("✨ 생성된 요약문:")
        print(summary_result)
        print("-" * 80)


In [5]:
if __name__ == '__main__':
    main(news_title = "한일 정상 “한미일 공조 발전…북한 포함 지정학적 위기대응”", news_document = """
주요 7개국(G7) 정상회의 참석을 위해 캐나다를 방문 중인 이재명 대통령이 17일 (현지시간) 캐내내스키스 현지에서 이시바 시게루 일본 총리와 정상회담을 진행했다.

대통령실에 따르면 양국 정상은 이날 오후 3시 30분부터 약 30분간 회담을 진행, 북한 문제 등 지역의 여러 지정학적 위기에 대응하기 위해 한미일 공조를 지속적으로 유지·발전시키고 한일간 협력도 강화하기로 했다.

두 정상은 급변하는 국제정세 속에 지역 내 평화와 안정을 유지하고 국익을 극대화하는 방안에 대해서도 깊이 있는 의견을 교환하고, 유사한 입장에 있는 양국이 더 긴밀히 협력을 모색해 나가기로 했다.

또 ‘셔틀외교’ 재개에 대한 의지를 재확인했으며, 이를 위해 당국 간 논의를 진전시켜 나가기로 했다고 대통령실은 전했다. 이와 함께 양 정상은 국교 정상화 60주년을 맞아 보다 견고하고 성숙한 한일관계의 기반을 조성해 나가기로 했다.

대통령실은 “양 정상이 상호 존중과 신뢰를 바탕으로 한일관계를 발전시켜 나가기 위해 당국 간 보다 활발히 대화를 이어 나가기로 하고, 경제·문화 등 여러 분야에서 상호 국익을 도모하기 위한 협력 방안을 계속 논의해 나갈 필요성에 공감했다”고 설명했다."""
)

🔑 Google API 키를 로드합니다...
동일 뉴스 기사에 대한 프롬프트별 요약 결과 비교
사용 모델: gemini-1.5-flash

--- [ 프롬프트 #1: 뉴스 에디터' (간결하고 핵심적인 요약) ] ---

✨ 생성된 요약문:
* 이재명 대통령과 기시다 후미오 일본 총리는 캐나다에서 한일 정상회담을 개최했다.
* 양국 정상은 북한을 포함한 지정학적 위기 대응을 위해 한미일 공조를 강화하기로 합의했다.
* 양국은 ‘셔틀외교’ 재개를 위한 당국 간 논의를 진전시키기로 했다.
* 양국 정상은 국교 정상화 60주년을 맞아 경제, 문화 등 다방면에서 협력을 확대하기로 했다.

--------------------------------------------------------------------------------

--- [ 프롬프트 #2: 정보 큐레이터(정보를 흥미롭게 재구성) ] ---

✨ 생성된 요약문:
---

### 📰 한일 정상, 한미일 공조 강화 및 북한 문제 공동 대응 합의

- **긴밀한 한일 협력 시대의 개막:**  급변하는 국제 정세 속에서,  한국과 일본 두 정상은 지역 안보와 상호 국익을 위해 전에 없던 수준의 긴밀한 협력을 약속했습니다.  이제 단순한 외교 관계를 넘어, 공동의 목표를 향한 협력의 새로운 장이 열리고 있습니다.

- **한미일 공조의 핵심 전략:** 북한의 위협에 효과적으로 대응하고, 역내 지정학적 위기를 관리하기 위해 한미일 3국 간의 공조 체계를 더욱 발전시키기로 결정했습니다. 이는 동북아시아 안보 지형에 중대한 변화를 예고하는 신호탄입니다.

- **'셔틀 외교' 재개 의지 확인:**  끊임없이 변화하는 국제 관계 속에서, 양국 정상은 소통 채널을 강화하기 위해 '셔틀 외교' 재개에 대한 확고한 의지를 재확인했습니다. 이는 한일 관계 개선의 중요한 발걸음이며, 향후 외교 관계 개선의 속도에 영향을 미칠 주요 요소입니다.

- **국교 정상화 60주년, 새로운 협력 시대 도래:**  60주년을 맞은 한일 